In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

In [38]:
directory = "data/census_data"

df = []

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    print
    if filename.endswith('.csv'):
        df_temp = pd.read_csv(os.path.join(directory,filename))
        df_temp = df_temp.reset_index()
        df_temp = df_temp.drop(df_temp.index[0])
        df_temp["Year_ID"] = filename[:4] + df_temp['GEO.id2']
        df_temp = df_temp.set_index("Year_ID")
        df.append(df_temp)
        
        

In [39]:
census_data = pd.concat(df)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [41]:
census_data

,GEO.display-label,GEO.id,GEO.id2,HC01_VC03,HC01_VC04,HC01_VC05,HC01_VC06,HC01_VC07,HC01_VC08,HC01_VC09,...,HC04_VC90,HC04_VC91,HC04_VC92,HC04_VC93,HC04_VC94,HC04_VC95,HC04_VC96,HC04_VC98,HC04_VC99,index
Year_ID,,,,,,,,,,,,,,,,,,,,,
201521001,"Adair County, Kentucky",0500000US21001,21001,7048,4707,1690,3741,1258,263,120,...,1.2,1.7,1.4,NaN,NaN,2.4,2.2,NaN,NaN,1
201521003,"Allen County, Kentucky",0500000US21003,21003,7774,5507,2453,4464,1870,313,199,...,1.4,1.6,1.4,NaN,NaN,2.6,2.1,NaN,NaN,2
201521005,"Anderson County, Kentucky",0500000US21005,21005,8446,6015,2654,4760,1902,332,203,...,1.6,1.7,1.7,NaN,NaN,1.8,2.7,NaN,NaN,3
201521007,"Ballard County, Kentucky",0500000US21007,21007,3288,2180,772,1766,546,103,24,...,1.9,1.9,1.5,NaN,NaN,2.1,2.3,NaN,NaN,4
201521009,"Barren County, Kentucky",0500000US21009,21009,16979,11755,4993,9004,3280,717,394,...,0.9,1.2,0.9,NaN,NaN,2.0,1.6,NaN,NaN,5
201521011,"Bath County, Kentucky",0500000US21011,21011,4297,2950,1168,2238,795,211,128,...,2.5,2.4,1.0,NaN,NaN,3.0,2.7,NaN,NaN,6
201521013,"Bell County, Kentucky",0500000US21013,21013,11088,7049,2848,4666,1758,541,181,...,1.2,1.0,0.9,NaN,NaN,2.5,1.4,NaN,NaN,7
201521015,"Boone County, Kentucky",0500000US21015,21015,44709,33557,16407,26190,12004,2218,1371,...,0.8,1.1,0.9,NaN,NaN,0.8,1.3,NaN,NaN,8
201521017,"Bourbon County, Kentucky",0500000US21017,21017,7867,5281,2332,3815,1547,390,128,...,1.4,1.6,1.1,NaN,NaN,2.0,2.0,NaN,NaN,9


In [59]:
drug_data = pd.read_excel('data/drug_data.xlsx', sheet_name = "Data")

In [60]:
drug_data["Year_ID"] = drug_data["YYYY"].map(str) + drug_data["FIPS_Combined"].map(str)
#drug_data = drug_data.set_index("Year_ID")

In [61]:
drug_keys = drug_data.SubstanceName.unique()

In [62]:
drug_data = drug_data.set_index(["Year_ID", "SubstanceName"])

In [73]:
#drug_data

In [74]:
#drug_data.xs('Morphine', level='SubstanceName')